# 3.数据预处理--处理缺失值

<p>现实数据集中经常会出现缺失值，Pandas提供两种类型的缺失值标签：None和NaN（Not a Number）。None为object类型，NaN为浮点型。</p>
<p>Pandas还提供了一些方法来发现、剔除和替换数据中的缺失值。</p>

* isnull(): 创建一个布尔类型的掩码标签缺失值。

* notnull(): 与isnull()操作相反。

* dropna(): 返回一个剔除了缺失值的数据副本。

* fillna(): 返回一个填充了缺失值的数据副本。

<font color=blue>1）判断缺失值</font>

In [5]:
import pandas as pd
import numpy as np
import os

data = pd.Series([1, np.nan, 'hello', None])
data.isnull()

0    False
1     True
2    False
3     True
dtype: bool

In [3]:
data.notnull()

0     True
1    False
2     True
3    False
dtype: bool

<font color=blue>2）筛选包含缺失值的数据

In [15]:
csv_file = open(os.getcwd() + os.path.sep + 'births.csv')
file = pd.read_csv(csv_file)

df = pd.DataFrame(file)
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 15547 entries, 0 to 15546
Data columns (total 5 columns):
year      15547 non-null int64
month     15547 non-null int64
day       15067 non-null float64
gender    15547 non-null object
births    15547 non-null int64
dtypes: float64(1), int64(3), object(1)
memory usage: 607.4+ KB


In [16]:
# drop_duplicates() 使一行中有多个缺失值的数据仍然只输出一行
na_data=df[df.isnull().values==True].drop_duplicates()
na_data.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 480 entries, 15067 to 15546
Data columns (total 5 columns):
year      480 non-null int64
month     480 non-null int64
day       0 non-null float64
gender    480 non-null object
births    480 non-null int64
dtypes: float64(1), int64(3), object(1)
memory usage: 22.5+ KB


<font color=blue>3）剔除缺失值

In [17]:
df_nona = df.dropna()
df_nona.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 15067 entries, 0 to 15066
Data columns (total 5 columns):
year      15067 non-null int64
month     15067 non-null int64
day       15067 non-null float64
gender    15067 non-null object
births    15067 non-null int64
dtypes: float64(1), int64(3), object(1)
memory usage: 706.3+ KB


默认情况下，dropna()会剔除<b>任何</b>包含缺失值的<b>整行</b>数据。

In [18]:
df_nona = df.dropna(axis=1)
df_nona.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 15547 entries, 0 to 15546
Data columns (total 4 columns):
year      15547 non-null int64
month     15547 non-null int64
gender    15547 non-null object
births    15547 non-null int64
dtypes: int64(3), object(1)
memory usage: 485.9+ KB


df.dropna(axis=1)和df.dropna(axis='columns')会剔除<b>任何</b>包含缺失值的<b>整列</b>数据。

可以通过设置 how 和 thresh 参数来设置剔除数据中包含缺失值的阈值。

In [20]:
df_nona = df.dropna(how='all')
df_nona.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 15547 entries, 0 to 15546
Data columns (total 5 columns):
year      15547 non-null int64
month     15547 non-null int64
day       15067 non-null float64
gender    15547 non-null object
births    15547 non-null int64
dtypes: float64(1), int64(3), object(1)
memory usage: 728.8+ KB


<font color=blue>4）填充缺失值</font>
<p>Pandas提供的 fillna() 方法可以用来填充缺失值，其中 method 参数（可取 ffill 和 bfill）可以指定填充的方式，ffill用前面的有效值填充，而bfill用后面的有效值填充。</p>

In [21]:
df_nona = df.fillna(0)
df_nona.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 15547 entries, 0 to 15546
Data columns (total 5 columns):
year      15547 non-null int64
month     15547 non-null int64
day       15547 non-null float64
gender    15547 non-null object
births    15547 non-null int64
dtypes: float64(1), int64(3), object(1)
memory usage: 607.4+ KB


对比填充前后数据行的变化：

In [39]:
na_idx = df[df.isnull().values==True].index.tolist()
len(na_idx)

480

In [30]:
df.loc[na_idx[0]]

year        1989
month          1
day          NaN
gender         F
births    156749
Name: 15067, dtype: object

In [27]:
df_nona.loc[na_idx[0]]

year        1989
month          1
day            0
gender         F
births    156749
Name: 15067, dtype: object

In [28]:
df_nona = df.fillna(method='ffill')
df_nona.loc[na_idx[0]]

year        1989
month          1
day           31
gender         F
births    156749
Name: 15067, dtype: object

In [29]:
df_nona = df.fillna(method='bfill')
df_nona.loc[na_idx[0]]

year        1989
month          1
day          NaN
gender         F
births    156749
Name: 15067, dtype: object

此时，仍然为NaN，这是因为从这行开始直到最后一行，该字段的值均为NaN，即后面没有有效值了。因此，使用bfill方法没有效果。因此，在填充后要注意检查结果，确定所有的NaN都被填充。

确认最后一条包含Nan的数据是否是数据集中的最后一行。

In [41]:
na_idx[-1]

15546

fillna()方法还可以通过axis参数指定从前一行（axis=0）或者前一列（axis=1）获取有效值

In [35]:
df_nona = df.fillna(method='ffill',axis=1)
df_nona.loc[na_idx[0]]

year        1989
month          1
day            1
gender         F
births    156749
Name: 15067, dtype: object

利用平均值填充也是常用的填充方式

In [47]:
fill = df['day'].mean()
df_nona = df.fillna(fill)
df_nona.loc[na_idx[0]]

year         1989
month           1
day       17.7699
gender          F
births     156749
Name: 15067, dtype: object